In [ ]:
from data_io import load_tiff_files

import pandas as pd
import numpy as np

In [ ]:
cell_meta_df = pd.read_csv('../data/cell_meta.csv')

cell_masks = load_tiff_files('../data/cell_masks')
compartment_masks = load_tiff_files('../data/compartment_masks')

loose_obj_num_masks = load_tiff_files('../data/obj_num_nerve_mask')
pred_obj_num_masks = load_tiff_files('../results/filtered_obj_num_nerve_mask')

In [ ]:
def quantify_tumor_stroma_overlap(compartment_mask, nerve_mask):
    """ Computes the fraction of tumor cell (1) and tumor stroma (0) pixels within each nerve region. """
    unique_nerves = np.unique(nerve_mask[nerve_mask > 0])
    if len(unique_nerves) == 0:
        return pd.DataFrame(columns=["nerve_id", "pred_n_stroma", "pred_n_tumor"])

    data = []

    for nerve_id in unique_nerves:
        nerve_pixels = (nerve_mask == nerve_id)  # Boolean mask for current nerve

        # Count pixels overlapping with tumor cell (1) and tumor stroma (0)
        stroma_pixels = np.sum(compartment_mask[nerve_pixels] == 0)
        tumor_pixels = np.sum(compartment_mask[nerve_pixels] == 1)

        total_pixels = tumor_pixels + stroma_pixels

        frac_stroma = stroma_pixels / total_pixels
        frac_tumor = tumor_pixels / total_pixels

        data.append([nerve_id, frac_stroma, frac_tumor])

    return pd.DataFrame(data, columns=["nerve_id", "pred_n_stroma", "pred_n_tumor"])

In [ ]:
def map_cell_type_distribution(cell_mask, nerve_mask, cell_meta_df):
    """ Computes the fraction of each cell type in the nerve regions. """

    cell_mask = (cell_mask > 0).astype(int)

    unique_nerves = np.unique(nerve_mask[nerve_mask > 0])
    data = []

    for nerve_id in unique_nerves:
        nerve_pixels = (nerve_mask == nerve_id)  # Boolean mask for current nerve region

        # Compute the number of pixels belonging to each cell type
        cell_type_counts = [np.sum(cell_mask[i, :, :][nerve_pixels]) for i in range(cell_mask.shape[0])]

        # Compute total nerve region size
        total_nerve_pixels = np.sum(nerve_pixels)
        total_assigned_pixels = sum(cell_type_counts)
        unclassified_pixels = total_nerve_pixels - total_assigned_pixels

        # Compute fraction of each cell type in the nerve region
        cell_type_fractions = [count / total_nerve_pixels for count in cell_type_counts]
        unclassified_fraction = unclassified_pixels / total_nerve_pixels

        data.append([nerve_id] + cell_type_fractions + [unclassified_fraction])

    # Use cell type names from cell_meta_df for column names
    col_names = ["nerve_id"] + sorted(cell_meta_df['cell_type'].unique()) + ["unclassified"]
    df = pd.DataFrame(data, columns=col_names)

    return df

In [ ]:
tumor_stroma_fractions = []
cell_type_fractions = []

for case_id, nerve_mask in pred_obj_num_masks.items():
    compartment_mask = compartment_masks[case_id]  # Single-layer mask for tumor/stroma separation
    cell_mask = cell_masks[case_id]  # Multi-layer mask for cell types

    tumor_stroma_df = quantify_tumor_stroma_overlap(compartment_mask, nerve_mask)
    tumor_stroma_df["case_id"] = case_id
    tumor_stroma_fractions.append(tumor_stroma_df)

    cell_fraction_df = map_cell_type_distribution(cell_mask, nerve_mask, cell_meta_df)
    cell_fraction_df["case_id"] = case_id
    cell_type_fractions.append(cell_fraction_df)

tumor_stroma_fractions_df = pd.concat(tumor_stroma_fractions, ignore_index=True)
cell_type_fractions_df = pd.concat(cell_type_fractions, ignore_index=True)

In [ ]:
# Summarize the tumor-stroma fractions and cell type distributions across all cases
tumor_stroma_aggregated = tumor_stroma_fractions_df.groupby("case_id").sum().drop("nerve_id", axis=1)
tumor_stroma_aggregated["pred_nerve_count"] = tumor_stroma_fractions_df.groupby("case_id").size()

cell_type_aggregated = cell_type_fractions_df.groupby("case_id").sum().drop("nerve_id", axis=1)
cell_type_aggregated["pred_nerve_count"] = cell_type_fractions_df.groupby("case_id").size()

In [ ]:
tumor_stroma_aggregated.to_csv("../results/tumor_stroma_fractions_aggregated.csv")
cell_type_aggregated.to_csv("../results/cell_type_fractions_aggregated.csv")

In [ ]:
tumor_stroma_fractions_df = pd.read_csv("../results/tumor_stroma_fractions_aggregated.csv")
cell_type_fractions_df = pd.read_csv("../results/cell_type_fractions_aggregated.csv")

In [ ]:
tumor_stroma_fractions_df

In [ ]:
cell_type_fractions_df